### Determining the optimal number of hidden layers and neurons for an Artificial Neural Network (ANN) 
This can be challenging and often requires experimentation. However, there are some guidelines and methods that can help you in making an informed decision:

- Start Simple: Begin with a simple architecture and gradually increase complexity if needed.
- Grid Search/Random Search: Use grid search or random search to try different architectures.
- Cross-Validation: Use cross-validation to evaluate the performance of different architectures.
- Heuristics and Rules of Thumb: Some heuristics and empirical rules can provide starting points, such as:
  -    The number of neurons in the hidden layer should be between the size of the input layer and the size of the output layer.
  -  A common practice is to start with 1-2 hidden layers.

In [49]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasRegressor
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras.callbacks import EarlyStopping
import pickle
import json

In [14]:
data=pd.read_csv('Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save encoders and scaler for later use
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [15]:
def create_model(neurons=32, layers=2):
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1],)))
    model.add(Dense(neurons, activation='relu'))

    for _ in range(layers - 1):
        model.add(Dense(neurons, activation='relu'))

    model.add(Dense(1))  # Output layer
    model.compile(optimizer='adam', loss="mean_absolute_error", metrics=['mae'])
    return model

In [55]:
## Create a Keras classifier
model=KerasRegressor(layers=2,neurons=32,model=create_model,verbose=1)

In [53]:
param_grid = {
    'neurons': [16, 32],
    'layers': [1, 2],
    'epochs': [25]
}

In [56]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=2, verbose=1)

# Fit the grid search
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
grid_result = grid.fit(X_train, y_train, callbacks=[early_stopping], validation_split=0.2)

# Print the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 2 folds for each of 4 candidates, totalling 8 fits
Epoch 1/25
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 99596.2344 - mae: 99596.2344 - val_loss: 102076.0078 - val_mae: 102076.0078
Epoch 2/25
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 100442.2109 - mae: 100442.2109 - val_loss: 101677.3828 - val_mae: 101677.3828
Epoch 3/25
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 98083.7188 - mae: 98083.7188 - val_loss: 100602.4297 - val_mae: 100602.4297
Epoch 4/25
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 97490.8672 - mae: 97490.8672 - val_loss: 98617.4297 - val_mae: 98617.4297
Epoch 5/25
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 95329.2734 - mae: 95329.2734 - val_loss: 95608.3516 - val_mae: 95608.3516
Epoch 6/25
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 91603.8359 - mae: 91603.8359 - val_loss: 91606.0234 - val_mae: 91606.0234
Epoch 7/25
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 89457.4844 - mae: 89457.4844 - val_loss: 86722.3828 - val_mae: 86722.

In [67]:
best_model = grid.best_estimator_.model_

best_model.save('optimized_model.keras')